In [2]:
import os
from loss import *
from config import opt
import models
import torch
import torch.optim as optim
from data.dataset import ImageDataSet, collate_fn, get_images, image_label
import numpy as np
from torch.utils.data import DataLoader
from torch import nn
from torch.autograd import Variable
from torch.optim import lr_scheduler
import data.dataset
import torch.utils.data as data
import time
import cv2

In [3]:
class ImageDataSet(data.Dataset):
    
    def __init__(self,img_root,txt_root):
        self.image_list,self.img_name = get_images(img_root)
        self.txt_root = txt_root

    def __getitem__(self, index):
        img,score_map,geo_map,training_mask = image_label(self.txt_root,self.image_list,self.img_name,index,input_size=512,random_scale=np.array([0.5,1,2.0,3.0]),background_ratio=3./8)
        return img,score_map,geo_map,training_mask


    def __len__(self):
        return len(self.image_list)

In [4]:
def collate_fn(batch):
    img, score_map, geo_map, training_mask = zip(*batch)
    bs = len(score_map)
    images = []
    score_maps = []
    geo_maps = []
    training_masks = []
    for i in range(bs):
        if img[i] is not None:
            a = torch.from_numpy(img[i])
            a = a.permute(2, 0, 1)
            images.append(a)
            b = torch.from_numpy(score_map[i])
            b = b.permute(2, 0, 1)
            score_maps.append(b)
            c = torch.from_numpy(geo_map[i])
            c = c.permute(2, 0, 1)
            geo_maps.append(c)
            d = torch.from_numpy(training_mask[i])
            d = d.permute(2, 0, 1)
            training_masks.append(d)
    images = torch.stack(images, 0)
    score_maps = torch.stack(score_maps, 0)
    geo_maps = torch.stack(geo_maps, 0)
    training_masks = torch.stack(training_masks, 0)

    return images, score_maps, geo_maps, training_masks

In [5]:
root_path = 'D:/_data/personal_project/ICDAR_2015/'
train_img = root_path + 'train_img/'
train_txt = root_path + 'train_gt/'

In [6]:
image_list, img_name = get_images(train_img)
img,score_map,geo_map,training_mask = image_label(train_txt, image_list, img_name, 1, input_size=512, random_scale=np.array([0.5,1,2.0,3.0]), background_ratio=3./8)

In [7]:
trainset = ImageDataSet(train_img, train_txt)

In [10]:
trainset[4]

(array([[[64., 59., 53.],
         [64., 59., 54.],
         [64., 59., 55.],
         ...,
         [86., 87., 91.],
         [85., 86., 89.],
         [85., 86., 88.]],
 
        [[65., 60., 55.],
         [65., 60., 55.],
         [65., 60., 56.],
         ...,
         [86., 87., 91.],
         [85., 86., 89.],
         [85., 86., 88.]],
 
        [[65., 60., 56.],
         [65., 60., 56.],
         [65., 60., 56.],
         ...,
         [86., 87., 91.],
         [86., 87., 89.],
         [85., 86., 89.]],
 
        ...,
 
        [[ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         ...,
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.]],
 
        [[ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         ...,
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.]],
 
        [[ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         ...,
         [ 0.,  0.,  0.],
  

In [1]:
root_path = 'D:/_data/personal_project/ICDAR_2015/'
train_img = root_path + 'train_img'
train_txt = root_path + 'train_gt'
